In [1]:
import numpy as np
import pandas as pd

n_splits = 5

In [2]:
ds_train = pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')
ds_test = pd.read_csv('../input/tabular-playground-series-apr-2021/test.csv')
dataset = pd.concat([ds_train, ds_test], axis=0)

PassengerId = ds_test['PassengerId'].values

dataset = dataset.drop(['PassengerId'], axis=1)

print(dataset.info())
dataset.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 0 to 99999
Data columns (total 11 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Survived  100000 non-null  float64
 1   Pclass    200000 non-null  int64  
 2   Name      200000 non-null  object 
 3   Sex       200000 non-null  object 
 4   Age       193221 non-null  float64
 5   SibSp     200000 non-null  int64  
 6   Parch     200000 non-null  int64  
 7   Ticket    190196 non-null  object 
 8   Fare      199733 non-null  float64
 9   Cabin     61303 non-null   object 
 10  Embarked  199473 non-null  object 
dtypes: float64(3), int64(3), object(5)
memory usage: 18.3+ MB
None


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1.0,1,"Oconnor, Frankie",male,NaN,2,0,209245,27.14,C12239,S
1,0.0,3,"Bryan, Drew",male,NaN,0,0,27323,13.35,NaN,S
2,0.0,3,"Owens, Kenneth",male,0.33,1,2,CA 457703,71.29,NaN,S
3,0.0,3,"Kramer, James",male,19.00,0,0,A. 10866,13.04,NaN,S
4,1.0,3,"Bond, Michael",male,25.00,0,0,427635,7.76,NaN,S


# Feature Engginering

In [3]:
## Cabin
# NaN means they didn't have a private cabin
dataset['Cabin'] = dataset['Cabin'].fillna('N')

# Retrive top alphabet
dataset['Cabin'] = dataset['Cabin'].map(lambda x: x[0])
dataset.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1.0,1,"Oconnor, Frankie",male,NaN,2,0,209245,27.14,C,S
1,0.0,3,"Bryan, Drew",male,NaN,0,0,27323,13.35,N,S
2,0.0,3,"Owens, Kenneth",male,0.33,1,2,CA 457703,71.29,N,S
3,0.0,3,"Kramer, James",male,19.00,0,0,A. 10866,13.04,N,S
4,1.0,3,"Bond, Michael",male,25.00,0,0,427635,7.76,N,S


In [4]:
## Ticket
import re

def strip_key_word(target=''):    
    return target.replace('.', '').replace(' ', '').replace('/', '')

dataset['Ticket'] = dataset['Ticket'].fillna('X').apply(lambda x: str(x).split()[0] if len(str(x).split()) > 1 else 'X')
dataset['Ticket'] = dataset['Ticket'].apply(lambda x: strip_key_word(x))

dataset.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1.0,1,"Oconnor, Frankie",male,NaN,2,0,X,27.14,C,S
1,0.0,3,"Bryan, Drew",male,NaN,0,0,X,13.35,N,S
2,0.0,3,"Owens, Kenneth",male,0.33,1,2,CA,71.29,N,S
3,0.0,3,"Kramer, James",male,19.00,0,0,A,13.04,N,S
4,1.0,3,"Bond, Michael",male,25.00,0,0,X,7.76,N,S


In [5]:
## Name
# Take only last name
dataset['Name'] = dataset['Name'].map(lambda x: x.split(',')[0])

dataset.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1.0,1,Oconnor,male,NaN,2,0,X,27.14,C,S
1,0.0,3,Bryan,male,NaN,0,0,X,13.35,N,S
2,0.0,3,Owens,male,0.33,1,2,CA,71.29,N,S
3,0.0,3,Kramer,male,19.00,0,0,A,13.04,N,S
4,1.0,3,Bond,male,25.00,0,0,X,7.76,N,S


In [6]:
## SibSp and Parch
#dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1 #Add one, it's ownself
#dataset['Alone'] = dataset['Family'].apply(lambda x: 0 if x >= 1 else 0)

#dataset = dataset.drop(['SibSp', 'Parch'], axis=1)
dataset.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1.0,1,Oconnor,male,NaN,2,0,X,27.14,C,S
1,0.0,3,Bryan,male,NaN,0,0,X,13.35,N,S
2,0.0,3,Owens,male,0.33,1,2,CA,71.29,N,S
3,0.0,3,Kramer,male,19.00,0,0,A,13.04,N,S
4,1.0,3,Bond,male,25.00,0,0,X,7.76,N,S


# Missing Value Engineering

### Original dataset info()
 0   Survived  100000 non-null  float64  
 1   Pclass    200000 non-null  int64    
 2   Sex       200000 non-null  object   
 3   Age       193221 non-null  float64  
 4   SibSp     200000 non-null  int64    
 5   Parch     200000 non-null  int64    
 6   Ticket    190196 non-null  object   
 7   Fare      199733 non-null  float64  
 8   Cabin     61303 non-null   object   
 9   Embarked  199473 non-null  object   

In [7]:
## Show missing value information
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 0 to 99999
Data columns (total 11 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Survived  100000 non-null  float64
 1   Pclass    200000 non-null  int64  
 2   Name      200000 non-null  object 
 3   Sex       200000 non-null  object 
 4   Age       193221 non-null  float64
 5   SibSp     200000 non-null  int64  
 6   Parch     200000 non-null  int64  
 7   Ticket    200000 non-null  object 
 8   Fare      199733 non-null  float64
 9   Cabin     200000 non-null  object 
 10  Embarked  199473 non-null  object 
dtypes: float64(3), int64(3), object(5)
memory usage: 18.3+ MB
None


In [8]:
## Don't use MICE for numerical columns
#  MICE uses linear-regression model, there might not be a linear relation between Age and Fare

#pclass_age = dataset[['Age', 'Pclass']].dropna().groupby('Pclass').mean().to_dict()
#dataset.fillna({'Age': dataset['Pclass'].apply(lambda x: pclass_age['Age'][x])}, inplace=True)
dataset.fillna({'Age': dataset['Age'].mean()}, inplace=True)

## Fare has a large skew
dataset.fillna({'Fare': dataset['Fare'].mean()}, inplace=True)
dataset['Fare'] = np.log(np.clip(dataset['Fare'], 1e-100, 1e+100))

## Fillna with mode in Embarked
dataset.fillna({'Embarked': dataset['Embarked'].mode().values[0]}, inplace=True)

print(dataset.info())
dataset.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 0 to 99999
Data columns (total 11 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Survived  100000 non-null  float64
 1   Pclass    200000 non-null  int64  
 2   Name      200000 non-null  object 
 3   Sex       200000 non-null  object 
 4   Age       200000 non-null  float64
 5   SibSp     200000 non-null  int64  
 6   Parch     200000 non-null  int64  
 7   Ticket    200000 non-null  object 
 8   Fare      200000 non-null  float64
 9   Cabin     200000 non-null  object 
 10  Embarked  200000 non-null  object 
dtypes: float64(3), int64(3), object(5)
memory usage: 18.3+ MB
None


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1.0,1,Oconnor,male,34.464565,2,0,X,3.301009,C,S
1,0.0,3,Bryan,male,34.464565,0,0,X,2.591516,N,S
2,0.0,3,Owens,male,0.330000,1,2,CA,4.266756,N,S
3,0.0,3,Kramer,male,19.000000,0,0,A,2.568022,N,S
4,1.0,3,Bond,male,25.000000,0,0,X,2.048982,N,S


# Encoding

In [9]:
dataset = pd.get_dummies(data=dataset, columns=['Pclass', 'Sex', 'Cabin', 'Embarked'], drop_first=True)

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
for col in ['Ticket', 'Name']:
    dataset[col] = encoder.fit_transform(dataset[col])

dataset.head()

,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Pclass_2,Pclass_3,Sex_male,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_N,Cabin_T,Embarked_Q,Embarked_S
0,1.0,17441,34.464565,2,0,35,3.301009,0,0,1,0,1,0,0,0,0,0,0,0,1
1,0.0,3063,34.464565,0,0,35,2.591516,0,1,1,0,0,0,0,0,0,1,0,0,1
2,0.0,17798,0.330000,1,2,7,4.266756,0,1,1,0,0,0,0,0,0,1,0,0,1
3,0.0,12742,19.000000,0,0,0,2.568022,0,1,1,0,0,0,0,0,0,1,0,0,1
4,1.0,2335,25.000000,0,0,35,2.048982,0,1,1,0,0,0,0,0,0,1,0,0,1


# Modeling

In [10]:
ds_train = dataset[dataset['Survived'].notnull()]
ds_test = dataset[dataset['Survived'].isnull()]
ds_test = ds_test.drop(columns=['Survived'], axis=1)

y = ds_train['Survived']
X = ds_train.drop(columns=['Survived'], axis=1)

print(ds_train.shape)
print(ds_test.shape)

(100000, 20)
(100000, 19)


In [11]:
X.head()

,Name,Age,SibSp,Parch,Ticket,Fare,Pclass_2,Pclass_3,Sex_male,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_N,Cabin_T,Embarked_Q,Embarked_S
0,17441,34.464565,2,0,35,3.301009,0,0,1,0,1,0,0,0,0,0,0,0,1
1,3063,34.464565,0,0,35,2.591516,0,1,1,0,0,0,0,0,0,1,0,0,1
2,17798,0.330000,1,2,7,4.266756,0,1,1,0,0,0,0,0,0,1,0,0,1
3,12742,19.000000,0,0,0,2.568022,0,1,1,0,0,0,0,0,0,1,0,0,1
4,2335,25.000000,0,0,35,2.048982,0,1,1,0,0,0,0,0,0,1,0,0,1


In [12]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

def stratified_lgb(X,y, params):
    kf = StratifiedKFold(n_splits=n_splits,random_state=42,shuffle=True)                  
    accuracy=[]   # list contains AUC for each fold  
    for tr_idx, te_idx in kf.split(X, y):
        X_tr, X_te = X.iloc[tr_idx], X.iloc[te_idx]
        y_tr, y_te = y.iloc[tr_idx], y.iloc[te_idx]
        lgb_classifier = lgb.LGBMClassifier(**params)
        lgb_classifier.fit(X_tr, y_tr, eval_set=[(X_te, y_te)], verbose=False, early_stopping_rounds=200, eval_metric=['binary_logloss'])
        accuracy.append(accuracy_score(y_te, lgb_classifier.predict(X_te))) 
    return np.mean(accuracy)

## LightGBM Classification

def objective(trial):
    params = {
            'objective': 'binary',
            'metric': 'binary_logloss',
            'n_estimators': 1000,
            'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
            'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
            'num_leaves': trial.suggest_int('num_leaves', 2, 256),
            'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
            'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
            'learning_rate': trial.suggest_loguniform('learning_rate', 1e-7, 1e-2)
        }

    return stratified_lgb(X, y, params)
    
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, n_jobs=-1)
lgb_best = study.best_params
print(lgb_best)

[I 2021-04-20 04:02:09,233] A new study created in memory with name: no-name-b50447f9-07a8-40ae-9c21-0be3cf7d0901
[I 2021-04-20 04:08:20,224] Trial 0 finished with value: 0.57226 and parameters: {'lambda_l1': 1.0370980954305355e-08, 'lambda_l2': 0.0069579386470232174, 'num_leaves': 76, 'feature_fraction': 0.6255867864476611, 'bagging_fraction': 0.6479467597025078, 'bagging_freq': 4, 'min_child_samples': 90, 'learning_rate': 2.7602142034217896e-06}. Best is trial 0 with value: 0.57226.
[I 2021-04-20 04:08:20,278] Trial 1 finished with value: 0.57226 and parameters: {'lambda_l1': 0.0007617919664430911, 'lambda_l2': 0.00211196521121931, 'num_leaves': 84, 'feature_fraction': 0.7121600731316651, 'bagging_fraction': 0.4277533849923478, 'bagging_freq': 4, 'min_child_samples': 9, 'learning_rate': 3.773233503489724e-05}. Best is trial 0 with value: 0.57226.
[I 2021-04-20 04:10:39,291] Trial 2 finished with value: 0.72654 and parameters: {'lambda_l1': 2.385233000504782e-08, 'lambda_l2': 2.187191

{'lambda_l1': 2.0694333309934295e-05, 'lambda_l2': 2.3957296175807625e-05, 'num_leaves': 15, 'feature_fraction': 0.6718510668996267, 'bagging_fraction': 0.9494082140859689, 'bagging_freq': 4, 'min_child_samples': 12, 'learning_rate': 0.008345437977664183}


In [13]:
## Predict
lgb_best['n_estimators'] = 10000
lgb_best['objective'] = 'binary'
lgb_best['metric'] = 'binary_logloss'

kf = StratifiedKFold(n_splits=n_splits,random_state=42,shuffle=True)                  
predict = pd.DataFrame()
n=0   
for tr_idx, te_idx in kf.split(X, y):
    X_tr, X_te = X.iloc[tr_idx], X.iloc[te_idx]
    y_tr, y_te = y.iloc[tr_idx], y.iloc[te_idx]
    lgb_classifier = lgb.LGBMClassifier(**lgb_best)
    lgb_classifier.fit(X_tr, y_tr, eval_set=[(X_te, y_te)], verbose=False, early_stopping_rounds=100, eval_metric='binary_logloss')
    y_pred = lgb_classifier.predict(ds_test)
    predict[n] = y_pred
    n+=1

result = np.round(predict.mode(axis=1).loc[:,0].values).astype(int)

output = pd.DataFrame({'PassengerId': PassengerId, 'Survived': result})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

[LightGBM] [Warning] lambda_l1 is set=2.0694333309934295e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.0694333309934295e-05
[LightGBM] [Warning] bagging_fraction is set=0.9494082140859689, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9494082140859689
[LightGBM] [Warning] lambda_l2 is set=2.3957296175807625e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.3957296175807625e-05
[LightGBM] [Warning] feature_fraction is set=0.6718510668996267, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6718510668996267
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] lambda_l1 is set=2.0694333309934295e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.0694333309934295e-05
[LightGBM] [Warning] bagging_fraction is set=0.9494082140859689, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9494082140859689
[LightGBM] [Warning]